### 隐马尔可夫链词性标注
*具体流程如下：

![title](./hmm4.png)
![title](./hmm1.png)
![title](./hmm2.png)
![title](./hmm3.png)

#### 导入需要的库

In [91]:
import nltk
import sys
from nltk.corpus import brown

#### 对词进行预处理，给brown中的句子加入start和end标记

In [92]:
brown_tags_words=[]
for sent in brown.tagged_sents():   #tagged_sents表示
    brown_tags_words.append(('START','START'))
    brown_tags_words.extend([(tag[:2],word) for (word,tag) in sent]) #list.extend()表示在已存在列表中添加新列表内容
    brown_tags_words.append(('END','END'))
# print(brown_tags_words[0:10])

#### 第一步：通过nltk自带函数，统计词库中单词与tag的关系$ P(w_{i}|t_{i})=\frac{count(w_{i},t_{i})}{count(t_{i})}$

In [93]:
 #nltk.ConditionalFreDist表示计算条件‘频率’分布，即count（wi，ti）（输入为元组）
cfd_tagwords = nltk.ConditionalFreqDist(brown_tags_words)

#nltk.ConditionalProbDist表示计算tag与words之间的条件‘概率’分布,即p（wi|ti）
cpd_tagwords = nltk.ConditionalProbDist(cfd_tagwords,nltk.MLEProbDist) 

# print(cpd_tagwords['NN'].prob('County')) 输出nn是county的概率
print("The probability of an adjective (JJ) being 'new' is", cpd_tagwords["JJ"].prob("new"))
print("The probability of a verb (VB) being 'duck' is", cpd_tagwords["VB"].prob("duck"))

The probability of an adjective (JJ) being 'new' is 0.01472344917632025
The probability of a verb (VB) being 'duck' is 6.042713350943527e-05


#### 第二步：计算$P(t_{i}|t_{i-1})=\frac{count(t_{i-1},t_{i})}{count(t_{i-1})}$

In [94]:
brown_tags = [tag for (tag,word) in brown_tags_words] #取出tag

In [95]:
#计算count(ti-1,ti)
cfd_tags = nltk.ConditionalFreqDist(nltk.bigrams(brown_tags)) #nltk.bigrams表示将一个列表前后两个一组的元组，排列成list
#计算p（ti|ti-1）
cpd_tags = nltk.ConditionalProbDist(cfd_tags,nltk.MLEProbDist)

#### 结果：

In [96]:
print("if we have just seen 'DT',the probability of 'NN' is ",cpd_tags['TD'].prob('NN'))
print("if we have just seen 'VB',the probability of 'JJ' is ",cpd_tags['VB'].prob('JJ'))
print("if we have just seen 'VB',the probability of 'NN' is ",cpd_tags['VB'].prob('NN'))

if we have just seen 'DT',the probability of 'NN' is  0
if we have just seen 'VB',the probability of 'JJ' is  0.03443483365273389
if we have just seen 'VB',the probability of 'NN' is  0.10970977711020183


#### 举例：‘i want to race',的tag表示为：'PP VB TO VB',看看他们的概率是多少
* 分解就是P(START) * P(PP|START) * P(I | PP) *P(VB | PP) * P(want | VB) * P(TO | VB) * P(to | TO) *P(VB | TO) * P(race | VB) *P(END | VB)

In [97]:
prob_tagsquence = cpd_tags['START'].prob('PP')*cpd_tagwords["PP"].prob("I") * \
    cpd_tags["PP"].prob("VB") * cpd_tagwords["VB"].prob("want") * \
    cpd_tags["VB"].prob("TO") * cpd_tagwords["TO"].prob("to") * \
    cpd_tags["TO"].prob("VB") * cpd_tagwords["VB"].prob("race") * \
    cpd_tags["VB"].prob("END")
print("The probability of the tag sequence 'START PP VB TO VB END'for 'i want to race' is:",prob_tagsquence)

The probability of the tag sequence 'START PP VB TO VB END'for 'i want to race' is: 1.0817766461150474e-14


### Viterbi实现
* 如果目前有句话，计算最符合的tag

In [98]:
#将tags集合化
distinct_tags = set(brown_tags)

#随意设置一句话
sentence = ['I','want','to','race']
senlen = len(sentence)

#初始化
viterbi = []
backpointer = []  #回溯器：从1循环到句子的总长n，即为i；把所有tagx前一个tag记下
first_viterbi = {}
first_backpointer = {}  

for tag in distinct_tags:
    if tag == 'START':  #'START'不记录跳过
        continue
    first_viterbi[tag] = cpd_tags['START'].prob(tag)*cpd_tagwords[tag].prob(sentence[0])
    first_backpointer[tag] = 'START'

print('first_viterbi:',first_viterbi) #第一个viterbi
print('\n')
print('first_backpointer:',first_backpointer) #第一个回溯点
print('\n')

#将之前的分别存入viterbi和backpointer两个变量
viterbi.append(first_viterbi)
backpointer.append(first_backpointer)

currbest = max(first_viterbi.keys(),key=lambda tag:first_viterbi[tag])  #max()返回最大项
print('word',"'"+sentence[0]+"'",'current best two_tag sequence:',first_backpointer[currbest],currbest) #目前最可能的tag
print('\n')
for wordindex in range(1,len(sentence)):
    this_viterbi = {}
    this_backpointer = {}
    prev_viterbi = viterbi[-1]
    for tag in distinct_tags:
        if tag == 'START':
            continue
        #假设目前单词为w，tag为x，前一个tag为y，为了找到最好的以y，x结尾的tag，则要最大化prev_viterbi[y]*p(x|y)p(w|x)
        best_previous =max(prev_viterbi.keys(),key=lambda prevtag:prev_viterbi[prevtag]*cpd_tags[prevtag].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex]))
        this_viterbi[tag]=prev_viterbi[best_previous]*cpd_tags[best_previous].prob(tag)*cpd_tagwords[tag].prob(sentence[wordindex])
        this_backpointer[tag]=best_previous
    currbest=max(this_viterbi.keys(),key=lambda tag:this_viterbi[tag])
    print('word',"'"+sentence[wordindex]+"'",'current best two_tag sequence:',this_backpointer[currbest],currbest)
    print('\n')
    viterbi.append(this_viterbi)
    backpointer.append(this_backpointer)
    
#找到以end结束的tag sequence
prev_viterbi = viterbi[-1]
#最大化prev_viterbi[y]*p(end|y)
best_previous = max(prev_viterbi.keys(),key=lambda prevtag:prev_viterbi[prevtag]*cpd_tags[prevtag].prob("END"))

#计算prob_tagsequence=prev_viterbi[best_y]*p(end|best_y)
prob_tagsequence = prev_viterbi[best_previous]*cpd_tags[best_previous].prob('END')

#存储为倒序，因此backpointer()也要倒过来
best_tagsequence = ['END',best_previous]
backpointer.reverse()

#回溯所有回溯点，最好的tag都对应存储在backpointer中，'end'结尾的最好的tag存储在best_previous中
current_best_tag = best_previous
for bp in backpointer:
    best_tagsequence.append(bp[current_best_tag])
    current_best_tag = bp[current_best_tag]
    
best_tagsequence.reverse()
print('the sentece was:',end=' ')
for w in sentence: print(w,end=' ')
print('\n')
print('the best tag sequence is:',end=' ')
for t in best_tagsequence: print(t,end=' ')
print('\n')
print('the probability of the best tag sequence is',prob_tagsequence)

first_viterbi: {'PP': 0.014930900689060006, 'NI': 3.3324520848931064e-07, 'EX': 0.0, 'CS': 0.0, ')-': 0.0, ',': 0.0, 'RP': 0.0, 'AT': 0.0, ')': 0.0, ',-': 0.0, 'AP': 0.0, 'CC': 0.0, 'IN': 0.0, 'NP': 1.7319067623793952e-06, 'END': 0.0, 'MD': 0.0, 'NR': 0.0, 'BE': 0.0, '*': 0.0, 'VB': 0.0, 'DO': 0.0, 'TO': 0.0, ':-': 0.0, '*-': 0.0, ':': 0.0, 'WR': 0.0, 'WP': 0.0, 'HV': 0.0, "'": 0.0, 'DT': 0.0, '.': 0.0, 'WQ': 0.0, 'UH': 0.0, 'PN': 0.0, '--': 0.0, 'CD': 0.0, '(': 0.0, '(-': 0.0, '``': 0.0, "''": 0.0, 'AB': 0.0, 'OD': 0.0, 'RN': 0.0, 'WD': 0.0, 'NN': 1.0580313619573935e-06, 'JJ': 0.0, 'RB': 0.0, 'FW': 0.0, '.-': 0.0, 'QL': 0.0}


first_backpointer: {'PP': 'START', 'NI': 'START', 'EX': 'START', 'CS': 'START', ')-': 'START', ',': 'START', 'RP': 'START', 'AT': 'START', ')': 'START', ',-': 'START', 'AP': 'START', 'CC': 'START', 'IN': 'START', 'NP': 'START', 'END': 'START', 'MD': 'START', 'NR': 'START', 'BE': 'START', '*': 'START', 'VB': 'START', 'DO': 'START', 'TO': 'START', ':-': 'START', '